## Q6: Cath errors

We want to safely convert a string into a float, int, or leave it as a string, depending on its contents.  As we've already seen, python provides `float()` and `int()` functions for this:

In [1]:
a = "2.0"
b = float(a)
print(b, type(b))

2.0 <class 'float'>


But these throw exceptions if the conversion is not possible

In [2]:
a = "this is a string"
b = float(a)

ValueError: could not convert string to float: 'this is a string'

In [3]:
a = "1.2345"
b = int(a)

ValueError: invalid literal for int() with base 10: '1.2345'

In [4]:
b = float(a)
print(b, type(b))

1.2345 <class 'float'>


Notice that an int can be converted to a float, but if you convert a float to an int, you risk losing significant digits.  A string cannot be converted to either.

### Your task

Write a function, `convert_type(a)` that takes a string `a`, and converts it to a float if it is a number with a decimal point, an int if it is an integer, or leaves it as a string otherwise, and returns the result.  You'll want to use exceptions to prevent the code from aborting.

### My solution:

In [1]:
def convert_type(a):
    """ converts a string 'a' to a float if it is 
    a number with a decimal point, to an int if it is
    an integer, or leaves it as a string otherwise. """
    try:
        return int(a)
    except ValueError:
        try:
            return float(a)
        except ValueError:
            return a        
        
print(type(convert_type("hello")))
print(type(convert_type("3.0")))
print(type(convert_type("12")))
print(type(convert_type("1.2")))
print(type(convert_type("agent.007")))
print(type(convert_type("21 guns")))
print(type(convert_type("1.0001")))

<class 'str'>
<class 'float'>
<class 'int'>
<class 'float'>
<class 'str'>
<class 'str'>
<class 'float'>


## Q8: Shopping cart

Let's write a simple shopping cart class -- this will hold items that you intend to purchase as well as the amount, etc.  And allow you to add / remove items, get a subtotal, etc.

We'll use two classes: `Item` will be a single item and `ShoppingCart` will be the collection of items you wish to purchase.

First, our store needs an inventory -- here's what we have for sale:

In [8]:
INVENTORY_TEXT = """
apple, 0.60
banana, 0.20
grapefruit, 0.75
grapes, 1.99
kiwi, 0.50
lemon, 0.20
lime, 0.25
mango, 1.50
papaya, 2.95
pineapple, 3.50
blueberries, 1.99
blackberries, 2.50
peach, 0.50
plum, 0.33
clementine, 0.25
cantaloupe, 3.25
pear, 1.25
quince, 0.45
orange, 0.60
"""

# this will be a global -- convention is all caps
INVENTORY = {}
for line in INVENTORY_TEXT.splitlines():
    if line.strip() == "":
        continue
    item, price = line.split(",")
    INVENTORY[item] = float(price)

In [9]:
INVENTORY

{'apple': 0.6,
 'banana': 0.2,
 'grapefruit': 0.75,
 'grapes': 1.99,
 'kiwi': 0.5,
 'lemon': 0.2,
 'lime': 0.25,
 'mango': 1.5,
 'papaya': 2.95,
 'pineapple': 3.5,
 'blueberries': 1.99,
 'blackberries': 2.5,
 'peach': 0.5,
 'plum': 0.33,
 'clementine': 0.25,
 'cantaloupe': 3.25,
 'pear': 1.25,
 'quince': 0.45,
 'orange': 0.6}

### Item 

Let's write an item class now -- we want it to hold the name and quantity.  

You should have the following features:

* The name should be something in our inventory

* Our shopping cart will include a list of all the items we want to buy, so we want to be able to check for duplicates.  Implement the equal test, `==`, using `__eq__`

* We'll want to consolidate duplicates, so implement the `+` operator, using `__add__` so we can add items together in our shopping cart.  Note, add should raise a ValueError if you try to add two `Items` that don't have the same name.

Here's a start:

In [10]:
class Item:
    """ an item to buy """
    
    def __init__(self, name, quantity=1):
        """keep track of an item that is in our inventory"""
        if name not in INVENTORY:
            raise ValueError("invalid item name: not in inventory")
        self.name = name
        self.quantity = quantity
        
    def __repr__(self):
        return "{}: {}".format(self.name, self.quantity)
        
    def __eq__(self, other):
        """check if the items have the same name"""
        return self.name == other.name
    
    def __add__(self, other):
        """add two items together if they are the same type"""
        if self.name == other.name:
            return Item(self.name, self.quantity + other.quantity)
        else:
            raise ValueError("names of the items don't match")

Here are some tests your code should pass:

In [11]:
a = Item("apple", 10)
b = Item("banana", 20)

In [12]:
c = Item("apple", 20)

In [13]:
# won't work
a + b

ValueError: names of the items don't match

In [14]:
# will work
a += c
print(a)

apple: 30


In [15]:
d = Item("dog")

ValueError: invalid item name: not in inventory

In [16]:
# should be False
a == b

False

In [17]:
# should be True -- they have the same name
a == c

True

How do they behave in a list?

In [18]:
items = []
items.append(a)
items.append(b)
items

[apple: 30, banana: 20]

In [19]:
# should be True -- they have the same name
c in items

True

### ShoppingCart

Now we want to create a shopping cart.  The main thing it will do is hold a list of items.

In [21]:
class ShoppingCart:
    
    def __init__(self):
        # the list of items we control
        self.items = []
        
    def subtotal(self):
        """ return a subtotal of our items """
        print("\n")
        for item in self.items:
            print(f"{item.name}, #{item.quantity}, {item.quantity * INVENTORY[item.name]:.2f} $")
        print("\n")

    def add(self, name, quantity):
        """ add an item to our cart -- if an item of the same name already
        exists, then increment the quantity.  Otherwise, add a new item
        to the cart with the desired quantity."""
        item = Item(name, quantity)
        if item in self.items:
            self.items[self.items.index(item)] += item
        else:
            self.items.append(item)
        
    def remove(self, name):
        """ remove all of item name from the cart """
        item = Item(name)
        if item not in self.items:
            raise ValueError("invalid item name to remove: not in cart")
        else:
            self.items.pop(self.items.index(item))
            
        
        
    def report(self):
        """ print a summary of the cart """
        for item in self.items:
            print(item)

Here are some tests

In [36]:
sc = ShoppingCart()
sc.add("orange", 19)
sc.add("apple", 2)
sc.add("kiwi", 26)
sc.add("plum", 14)
sc.subtotal()
sc.report()



orange, #19, 11.40 $
apple, #2, 1.20 $
kiwi, #26, 13.00 $
plum, #14, 4.62 $


orange: 19
apple: 2
kiwi: 26
plum: 14


In [37]:
sc.add("apple", 9)

In [38]:
# apple should only be listed once in the report, with a quantity of 11
sc.report()

orange: 19
apple: 11
kiwi: 26
plum: 14


In [39]:
sc.subtotal()



orange, #19, 11.40 $
apple, #11, 6.60 $
kiwi, #26, 13.00 $
plum, #14, 4.62 $




In [40]:
sc.remove("apple")

# apple should no longer be listed
sc.subtotal()
sc.report()



orange, #19, 11.40 $
kiwi, #26, 13.00 $
plum, #14, 4.62 $


orange: 19
kiwi: 26
plum: 14
